In [3]:
""" speech_nodes.csv file contains words and their corresponding parts of speech, 
and the speech_edges.csv file contains connections between them.
"""
# import pandas library
import pandas as pd
#read csv files into respective DataFrames:
edges = pd.read_csv(r"H:\Data\speech_edges.csv")
nodes = pd.read_csv(r"H:\Data\speech_nodes.csv")
# see the DataFrames' data
edges.head()
nodes.head()

,name,part_of_speech
0,agreeable,adjective
1,man,noun
2,old,adjective
3,person,noun
4,anything,noun


In [5]:
# Mark all words in edges DataFrame with their parts of speech.
#First of all, we need to merge our tables on words which the edges begin with:
from_tab = pd.merge(nodes, edges,left_on='name', right_on='from')[['part_of_speech','from','to']]

In [7]:
#The next step is to merge tables on words which the edge goes to:
to_tab = pd.merge(nodes, edges, left_on='name', right_on='to')[['part_of_speech','to','from']]

In [8]:
#Let’s rename columns to exclude duplication (“pos” means part of speech):
from_tab.rename(columns = {"part_of_speech": "pos_from"}, inplace = True)
to_tab.rename(columns = {"part_of_speech": "pos_to" }, inplace = True)

In [9]:
# Finally, let’s merge tables to form the edges
tab = pd.merge(from_tab, to_tab, on=["to","from"])[["pos_from","from","pos_to","to"]]

In [10]:
#To see the first 5 lines of the table 
tab.head(5)

,pos_from,from,pos_to,to
0,noun,man,adjective,old
1,noun,man,noun,person
2,noun,man,noun,anything
3,adjective,old,noun,person
4,adjective,old,adjective,bad


In [15]:
# Connect and load data to the Neo4j graph database
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase
uri = "neo4j://localhost:7687"
graph = Graph(uri, auth=("neo4j", "password"))

In [17]:
# Create nodes and edges (relationships) and merge them into the previously created empty graph in Neo4j database:
#Iterate in row of "tab" which is created above
for row in tab.iterrows():
  #Create node_a and node_b then connect them with Relationship "with"
  node_a = Node(row[1]["pos_from"], name=row[1]["from"])
  node_b = Node(row[1]["pos_to"], name=row[1]["to"])
  rel = Relationship(node_a, "with", node_b)
  graph.merge(rel, "with", "name")

# After above connection, see data graph model through Neo4j Browser of Neo4j Desktop application or the link http://localhost:7474/browser/
# In Neo4j Browser interface, at Neo4j$ line type following command "MATCH p=()-[r:with]->() RETURN p LIMIT 100" to see 67 nodes and 100 relationships of the graph